### Transfer Learning - Leverage Pretrained Model
https://lightning.ai/courses/deep-learning-fundamentals/unit-7-overview-getting-started-with-computer-vision/unit-7.6-leveraging-pretrained-models-with-transfer-learning/

In [1]:
import torch
import torchvision
import lightning
from lightning.pytorch.loggers import CSVLogger
from torchvision import datasets, transforms
from torch.utils.data import dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from common_def import LightningModel, plot_metrics_from_csv_log
import torch.nn.functional as F

#### Load the pretrained ResNet model

In [2]:
models = torch.hub.list('pytorch/vision')
for model in models:
    if 'resnet' in model:
        print(model)

deeplabv3_resnet101
deeplabv3_resnet50
fcn_resnet101
fcn_resnet50
resnet101
resnet152
resnet18
resnet34
resnet50
wide_resnet101_2
wide_resnet50_2


Using cache found in /home/tu/.cache/torch/hub/pytorch_vision_main


In [3]:
resnet18_imagenet1kv1 = torch.hub.load('pytorch/vision', 'resnet18', weights='IMAGENET1K_V1')
resnet18_imagenet1kv1


Using cache found in /home/tu/.cache/torch/hub/pytorch_vision_main


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### Freeze layers and add Output Layer

In [4]:
# freeze all layers
# in case we want to update the whole model, just ignore this
for param in resnet18_imagenet1kv1.parameters():
    param.requires_grad = False

# change output layer
# original: (fc): Linear(in_features=512, out_features=1000, bias=True)
# our task requires 10 output classes
resnet18_imagenet1kv1.fc = torch.nn.Linear(512, 10)